## Data 분석 실습 (Kmong)

In [1]:
import pandas as pd

# DataFrame을 출력시 최대 출력할 수 있는 컬럼을 100개로 만들기
pd.options.display.max_columns = 100

### Data 불러오기

In [2]:
raw_log = pd.read_csv("data/kmong-conversion.csv")

print(raw_log.shape)

raw_log.head()

(434244, 19)


,eventcategory,isfirstactivity,apppackagename,appversion,devicetype,devicemanufacturer,osversion,canonicaldeviceuuid,sourcetype,channel,params_campaign,params_medium,params_term,inappeventcategory,inappeventlabel,eventdatetime,rowuuid,isfirstgoalactivity,event_rank
0,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,unattributed,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28T00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,True
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,samsung,Android7.0,8a871e50-0717-4aed-9bad-04ac3c3793be,unattributed,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28T00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,True
2,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS12.0,A9E5778A-8F3D-4597-9718-74BF953A9F64,unattributed,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28T00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,True
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,168761CB-CB67-4592-867D-52780D651297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28T00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,True
4,goal,False,com.kmong.iOS,4.0.4,iPhone,Apple,iOS11.4.1,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,viral,WEB,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28T00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,True


In [3]:
raw_funnel = pd.read_csv("data/kmong-funnel.csv")

print(raw_funnel.shape)

raw_funnel.head()

(53, 6)


,Lv2,viewid,viewid desc,Lv1,funnel name,funnel desc
0,1100,home,홈 (탭),11,home,홈
1,1210,category_list,카테고리 목록 (탭),12,category,카테고리
2,1200,category_gig,카테고리-상품목록,12,category,카테고리
3,1300,search,검색,13,search,검색
4,1301,search_gig,검색-상품목록,13,search,검색


In [4]:
raw_category = pd.read_csv("data/kmong-category.csv")

print(raw_category.shape)

raw_category.head()

(245, 9)


,depth,categoryid,categoryname,cat1_id,cat2_id,cat3_id,cat1,cat2,cat3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN
1,1,2,마케팅,2,NaN,NaN,마케팅,NaN,NaN
2,1,3,번역·통역,3,NaN,NaN,번역·통역,NaN,NaN
3,1,4,문서작성,4,NaN,NaN,문서작성,NaN,NaN
4,1,6,IT·프로그래밍,6,NaN,NaN,IT·프로그래밍,NaN,NaN


### Data 정리하기

#### 1. 알아보기 쉬운 column 명으로 수정

In [5]:
raw_log["userid"] = raw_log["canonicaldeviceuuid"]

print(raw_log.shape)

raw_log[["canonicaldeviceuuid", "userid"]].head()

(434244, 20)


,canonicaldeviceuuid,userid
0,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957
1,8a871e50-0717-4aed-9bad-04ac3c3793be,8a871e50-0717-4aed-9bad-04ac3c3793be
2,A9E5778A-8F3D-4597-9718-74BF953A9F64,A9E5778A-8F3D-4597-9718-74BF953A9F64
3,168761CB-CB67-4592-867D-52780D651297,168761CB-CB67-4592-867D-52780D651297
4,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D


#### 2. datetime 형태로 추출

In [6]:
raw_log["eventdatetime"] = pd.to_datetime(raw_log["eventdatetime"])

raw_log["eventdatetime"].head()

0   2018-09-28 00:00:00+09:00
1   2018-09-28 00:00:00+09:00
2   2018-09-28 00:00:00+09:00
3   2018-09-28 00:00:01+09:00
4   2018-09-28 00:00:02+09:00
Name: eventdatetime, dtype: datetime64[ns, pytz.FixedOffset(540)]

#### 2-1. 년, 일, 시, 분, 초 로 나눠서 표시

In [7]:
raw_log["eventdatetime_year"] = raw_log["eventdatetime"].dt.year
raw_log["eventdatetime_month"] = raw_log["eventdatetime"].dt.month
raw_log["eventdatetime_day"] = raw_log["eventdatetime"].dt.day
raw_log["eventdatetime_hour"] = raw_log["eventdatetime"].dt.hour
raw_log["eventdatetime_minute"] = raw_log["eventdatetime"].dt.minute
raw_log["eventdatetime_second"] = raw_log["eventdatetime"].dt.second

print(raw_log.shape)

raw_log[["eventdatetime", "eventdatetime_year", "eventdatetime_month", "eventdatetime_day", "eventdatetime_hour", "eventdatetime_minute", "eventdatetime_second"]].head()

(434244, 26)


,eventdatetime,eventdatetime_year,eventdatetime_month,eventdatetime_day,eventdatetime_hour,eventdatetime_minute,eventdatetime_second
0,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0
1,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0
2,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0
3,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1
4,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2


#### 3. os 버전 나누기

#### 3-1. os 명으로 나누기

In [8]:
raw_log["osversion"].head()

0     iOS11.4.1
1    Android7.0
2       iOS12.0
3     iOS11.4.1
4     iOS11.4.1
Name: osversion, dtype: object

In [9]:
def find_ostype(osversion):
    if "iOS" in osversion:
        return "iOS"
    else:
        return "Android"

In [10]:
find_ostype("iOS12.0")

'iOS'

In [11]:
find_ostype("Android7.0")

'Android'

In [12]:
raw_log["ostype(clean)"] = raw_log["osversion"].apply(find_ostype)

print(raw_log.shape)

raw_log[["osversion", "ostype(clean)"]].head()

(434244, 27)


,osversion,ostype(clean)
0,iOS11.4.1,iOS
1,Android7.0,Android
2,iOS12.0,iOS
3,iOS11.4.1,iOS
4,iOS11.4.1,iOS


#### 3-2. os 버전 별로 나누기

In [13]:
def find_osversion(osversion):
    return osversion.replace("iOS", "").replace("Android", "")

In [14]:
find_osversion("iOS12.0")

'12.0'

In [15]:
find_osversion("Android7.0")

'7.0'

In [16]:
raw_log["osversion(clean)"] = raw_log["osversion"].apply(find_osversion)

print(raw_log.shape)

raw_log[["osversion", "osversion(clean)"]].head()

(434244, 28)


,osversion,osversion(clean)
0,iOS11.4.1,11.4.1
1,Android7.0,7.0
2,iOS12.0,12.0
3,iOS11.4.1,11.4.1
4,iOS11.4.1,11.4.1


In [17]:
raw_log[["osversion", "ostype(clean)", "osversion(clean)"]].head()

,osversion,ostype(clean),osversion(clean)
0,iOS11.4.1,iOS,11.4.1
1,Android7.0,Android,7.0
2,iOS12.0,iOS,12.0
3,iOS11.4.1,iOS,11.4.1
4,iOS11.4.1,iOS,11.4.1


#### 3-3. 다르게 정리 하는 법

In [18]:
raw_log.loc[raw_log["osversion"].str.contains("iOS"), "ostype(clean)"] = "iOS"

raw_log.loc[raw_log["osversion"].str.contains("Android"), "ostype(clean)"] = "Android"

raw_log["osversion(clean)"] = raw_log["osversion"].str.replace("iOS|Android", "")

print(raw_log.shape)

raw_log[["osversion", "ostype(clean)", "osversion(clean)"]].head()

(434244, 28)


,osversion,ostype(clean),osversion(clean)
0,iOS11.4.1,iOS,11.4.1
1,Android7.0,Android,7.0
2,iOS12.0,iOS,12.0
3,iOS11.4.1,iOS,11.4.1
4,iOS11.4.1,iOS,11.4.1


#### 4. 제조사 컬럼 정보를 정리

In [19]:
def clean_devicemanufacturer(devicemanufacturer):
    if "samsung" in devicemanufacturer:
        return "Samsung"
    elif "LGE" in devicemanufacturer:
        return "LG"
    elif "LG Electronics" in devicemanufacturer:
        return "LG"
    elif "Apple" in devicemanufacturer:
        return "Apple"
    else:
        return "Others"

In [20]:
clean_devicemanufacturer("samsung")

'Samsung'

In [21]:
clean_devicemanufacturer("LGE")

'LG'

In [22]:
clean_devicemanufacturer("Hwawei")

'Others'

In [23]:
raw_log["devicemanufacturer(clean)"] = raw_log["devicemanufacturer"].apply(clean_devicemanufacturer)

print(raw_log.shape)

raw_log[["devicemanufacturer", "devicemanufacturer(clean)"]].head()

(434244, 29)


,devicemanufacturer,devicemanufacturer(clean)
0,Apple,Apple
1,samsung,Samsung
2,Apple,Apple
3,Apple,Apple
4,Apple,Apple


In [24]:
raw_log["devicemanufacturer(clean)"].value_counts()

Samsung    230313
Apple      163566
LG          32649
Others       7716
Name: devicemanufacturer(clean), dtype: int64

#### 4-1. 다르게 정리하는 방법

In [25]:
raw_log["clean_devicemanufacturer"] = raw_log["devicemanufacturer"].replace("samsung", "Samsung")\
                                .replace("LG Electronics", "LG")\
                                .replace("LGE", "LG")

raw_log["clean_devicemanufacturer"].head()

raw_log.loc[~raw_log["devicemanufacturer(clean)"].isin(["Apple", "Samsung", "LG"]), "devicemanufacturer(clean)"] = "Others"

print(raw_log.shape)

(434244, 30)


In [26]:
raw_log["devicemanufacturer(clean)"].value_counts()

Samsung    230313
Apple      163566
LG          32649
Others       7716
Name: devicemanufacturer(clean), dtype: int64

#### 5. channel 컬럼 정리

In [27]:
raw_log["channel"].head()

0    unattributed
1    unattributed
2    unattributed
3             NaN
4             WEB
Name: channel, dtype: object

In [28]:
raw_log["channel"].isnull().head()

0    False
1    False
2    False
3     True
4    False
Name: channel, dtype: bool

In [29]:
raw_log["channel"].value_counts()

unattributed            270834
WEB                      27639
google-play              19973
google.adwords            4076
google                    3186
(not set)                 2444
m_daum                     966
m_naver                    849
apple.searchads            697
pc_naver                   227
m_naverpowercontents       148
facebook                   126
Name: channel, dtype: int64

In [30]:
import numpy as np

def clean_channel(channel):
    if pd.isnull(channel):
        return np.nan
    if "google" in channel:
        return "google"
    elif "naver" in channel:
        return "naver"
    elif "daum" in channel:
        return "daum"
    elif "apple" in channel:
        return "apple"
    elif "WEB" in channel:
        return "web"
    else:
        return channel

In [31]:
clean_channel("google.dfaf")

'google'

In [32]:
clean_channel("(not set)")

'(not set)'

In [33]:
raw_log["channel(clean)"] = raw_log["channel"].apply(clean_channel)

print(raw_log.shape)

raw_log["channel(clean)"].value_counts()

(434244, 31)


unattributed    270834
web              27639
google           27235
(not set)         2444
naver             1224
daum               966
apple              697
facebook           126
Name: channel(clean), dtype: int64

#### 5-1. 다른 방식으로 정리

In [34]:
raw_log["channel(clean)"] = raw_log["channel"].replace("google-play", "google") \
                                              .replace("google.adwords", "google") \
                                              .replace("m_naver", "naver") \
                                              .replace("pc_naver", "naver") \
                                              .replace("m_naverpowercontents", "naver") \
                                              .replace("m_daum", "daum") \
                                              .replace("apple.searchads", "apple") \
                                              .replace("WEB", "web") \

print(raw_log.shape)

raw_log["channel(clean)"].value_counts()

(434244, 31)


unattributed    270834
web              27639
google           27235
(not set)         2444
naver             1224
daum               966
apple              697
facebook           126
Name: channel(clean), dtype: int64

#### 6. inappeventcategory 컬럼을 나누어주기

In [35]:
raw_log["inappeventcategory"].head()

0                 home.view
1           gig_detail.view
2         inbox_detail.view
3                       NaN
4    buyer_order_track.view
Name: inappeventcategory, dtype: object

In [36]:
def viewcategory(inappeventcategory):
    if pd.isnull(inappeventcategory):
        return np.nan
    else:
        return inappeventcategory.split(".")[0].split("_")[0]

In [37]:
viewcategory(pd.np.nan)

nan

In [38]:
viewcategory("gig_detail.view")

'gig'

In [39]:
def viewid(inappeventcategory):
    if pd.isnull(inappeventcategory):
        return pd.np.nan
    else:
        return inappeventcategory.split(".")[0]

In [40]:
viewid("gig_detail.view")

'gig_detail'

In [41]:
def viewaction(inappeventcategory):
    if pd.isnull(inappeventcategory):
        return np.nan
    else:
        return inappeventcategory.split(".")[1]

In [42]:
viewaction("gig_detail.view")

'view'

In [43]:
raw_log["viewcategory"] = raw_log["inappeventcategory"].apply(viewcategory)
raw_log["viewid"] = raw_log["inappeventcategory"].apply(viewid)
raw_log["viewaction"] = raw_log["inappeventcategory"].apply(viewaction)

print(raw_log.shape)

raw_log[["inappeventcategory", "viewcategory", "viewid", "viewaction"]].head()

(434244, 34)


,inappeventcategory,viewcategory,viewid,viewaction
0,home.view,home,home,view
1,gig_detail.view,gig,gig_detail,view
2,inbox_detail.view,inbox,inbox_detail,view
3,NaN,NaN,NaN,NaN
4,buyer_order_track.view,buyer,buyer_order_track,view


#### 7. 필요하지 않은 컬럼을 버리기

In [44]:
# 필요없는 컬럼명을 직접 입력

names = ["osversion", "devicemanufacturer", "canonicaldeviceuuid", "channel", "event_rank"]

dropped_log = raw_log.drop(names, axis = 1).copy()

print(dropped_log.shape)

dropped_log.head()

(434244, 29)


,eventcategory,isfirstactivity,apppackagename,appversion,devicetype,sourcetype,params_campaign,params_medium,params_term,inappeventcategory,inappeventlabel,eventdatetime,rowuuid,isfirstgoalactivity,userid,eventdatetime_year,eventdatetime_month,eventdatetime_day,eventdatetime_hour,eventdatetime_minute,eventdatetime_second,ostype(clean),osversion(clean),devicemanufacturer(clean),clean_devicemanufacturer,channel(clean),viewcategory,viewid,viewaction
0,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28 00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018,9,28,0,0,0,iOS,11.4.1,Apple,Apple,unattributed,home,home,view
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28 00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,8a871e50-0717-4aed-9bad-04ac3c3793be,2018,9,28,0,0,0,Android,7.0,Samsung,Samsung,unattributed,gig,gig_detail,view
2,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28 00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018,9,28,0,0,0,iOS,12.0,Apple,Apple,unattributed,inbox,inbox_detail,view
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28 00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,168761CB-CB67-4592-867D-52780D651297,2018,9,28,0,0,1,iOS,11.4.1,Apple,Apple,NaN,NaN,NaN,NaN
4,goal,False,com.kmong.iOS,4.0.4,iPhone,viral,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28 00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018,9,28,0,0,2,iOS,11.4.1,Apple,Apple,web,buyer,buyer_order_track,view


#### 7-1 다른 방법으로 없애기

In [45]:
# 있어야 하는 컬럼명을 직접 입력

need_columns = ['eventcategory', 'isfirstactivity', 'apppackagename', 'appversion',
               'devicetype', 'sourcetype', 'params_campaign', 'params_medium', 'params_term',
               'inappeventcategory', 'inappeventlabel', 'eventdatetime',
               'rowuuid', 'isfirstgoalactivity', 'ostype(clean)',
               'osversion(clean)', 'devicemanufacturer(clean)', 'userid', 'channel(clean)',
               'eventdatetime_year', 'eventdatetime_month', 'eventdatetime_day',
               'eventdatetime_hour', 'eventdatetime_minute', 'eventdatetime_second',
               'viewid', 'viewaction', 'viewcategory']

dropped_log1 = raw_log[need_columns].copy()

print(dropped_log1.shape)

dropped_log1.head()

(434244, 28)


,eventcategory,isfirstactivity,apppackagename,appversion,devicetype,sourcetype,params_campaign,params_medium,params_term,inappeventcategory,inappeventlabel,eventdatetime,rowuuid,isfirstgoalactivity,ostype(clean),osversion(clean),devicemanufacturer(clean),userid,channel(clean),eventdatetime_year,eventdatetime_month,eventdatetime_day,eventdatetime_hour,eventdatetime_minute,eventdatetime_second,viewid,viewaction,viewcategory
0,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28 00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,iOS,11.4.1,Apple,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,unattributed,2018,9,28,0,0,0,home,view,home
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28 00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,Android,7.0,Samsung,8a871e50-0717-4aed-9bad-04ac3c3793be,unattributed,2018,9,28,0,0,0,gig_detail,view,gig
2,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28 00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,iOS,12.0,Apple,A9E5778A-8F3D-4597-9718-74BF953A9F64,unattributed,2018,9,28,0,0,0,inbox_detail,view,inbox
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28 00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,iOS,11.4.1,Apple,168761CB-CB67-4592-867D-52780D651297,NaN,2018,9,28,0,0,1,NaN,NaN,NaN
4,goal,False,com.kmong.iOS,4.0.4,iPhone,viral,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28 00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,iOS,11.4.1,Apple,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,web,2018,9,28,0,0,2,buyer_order_track,view,buyer


#### 8. 이름을 통일성있게 변경

In [46]:
dropped_log.rename(columns= {"eventcategory" : "event_category", 
                             "isfirstactivity" : "is_first_activity", 
                             "apppackagename" : "app_package_name",
                             "appversion": "app_version",
                             "devicetype":"device_type",
                             "sourcetype":"source_type",
                             "inappeventlabel":"in_app_event_category",
                             "inappeventlabel":"in_app_event_label",
                             "eventdatetime":"event_datetime",
                             "rowuuid":"row_uuid",
                             "isfirstgoalactivity":"is_first_goal_activity",
                             "userid":"user_id",
                             "eventdatetime_year":"event_datetime_year",
                             "eventdatetime_month":"event_datetime_month",
                             "eventdatetime_day":"event_datetime_day",
                             "eventdatetime_hour":"event_datetime_hour",
                             "eventdatetime_minute":"event_datetime_minute",
                             "eventdatetime_second":"event_datetime_second",
                             "ostype(clean)":"os_type",
                             "osversion(clean)":"os_version",
                             "devicemanufacturer(clean)":"device_manufacturer",
                             "channel(clean)":"channel",
                             "viewcategory":"view_category",
                             "viewid":"view_id",
                             "viewaction":"view_action",
                            }, inplace=True )

print(dropped_log.shape)

dropped_log.head()

(434244, 29)


,event_category,is_first_activity,app_package_name,app_version,device_type,source_type,params_campaign,params_medium,params_term,inappeventcategory,in_app_event_label,event_datetime,row_uuid,is_first_goal_activity,user_id,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,os_type,os_version,device_manufacturer,clean_devicemanufacturer,channel,view_category,view_id,view_action
0,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,home.view,NaN,2018-09-28 00:00:00+09:00,fd2a188c-bc9b-4702-9c47-b546b2614817,False,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018,9,28,0,0,0,iOS,11.4.1,Apple,Apple,unattributed,home,home,view
1,goal,False,com.kmong.kmong,3.3.5,SM-N935S,unattributed,NaN,NaN,NaN,gig_detail.view,41201.0,2018-09-28 00:00:00+09:00,e62dccef-dd70-4415-8a33-c8324ddaed38,False,8a871e50-0717-4aed-9bad-04ac3c3793be,2018,9,28,0,0,0,Android,7.0,Samsung,Samsung,unattributed,gig,gig_detail,view
2,goal,False,com.kmong.iOS,4.0.4,iPhone,unattributed,NaN,NaN,NaN,inbox_detail.view,NaN,2018-09-28 00:00:00+09:00,14eb3197-db83-493a-b7be-83582960c40b,False,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018,9,28,0,0,0,iOS,12.0,Apple,Apple,unattributed,inbox,inbox_detail,view
3,foreground,NaN,com.kmong.iOS,4.0.4,iPhone,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-28 00:00:01+09:00,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,NaN,168761CB-CB67-4592-867D-52780D651297,2018,9,28,0,0,1,iOS,11.4.1,Apple,Apple,NaN,NaN,NaN,NaN
4,goal,False,com.kmong.iOS,4.0.4,iPhone,viral,NaN,NaN,NaN,buyer_order_track.view,NaN,2018-09-28 00:00:02+09:00,236e9946-7801-4898-b609-06c8ab1139dc,False,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018,9,28,0,0,2,iOS,11.4.1,Apple,Apple,web,buyer,buyer_order_track,view


#### 9. 컬럼을 정렬

In [47]:
# Write your code here!
dropped_log = pd.DataFrame(dropped_log, columns = ["row_uuid", "app_package_name", "user_id", "event_datetime", 
                                                "event_datetime_year", "event_datetime_month", "event_datetime_day", "event_datetime_hour",
                                                "event_datetime_minute", "event_datetime_second", "device_manufacturer", "device_type",
                                                "os_type", "os_version", "app_version", "event_category", "view_category", "view_id",
                                                "view_action", "in_app_event_category", "in_app_event_label", "source_type", "channel",
                                                "params_campaign", "params_medium", "params_term", "is_first_activity", "is_first_goal_activity"])

print(dropped_log.shape)

dropped_log.head()

(434244, 28)


,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,NaN,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,NaN,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False
2,14eb3197-db83-493a-b7be-83582960c40b,com.kmong.iOS,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,12.0,4.0.4,goal,inbox,inbox_detail,view,NaN,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False
3,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,com.kmong.iOS,168761CB-CB67-4592-867D-52780D651297,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1,Apple,iPhone,iOS,11.4.1,4.0.4,foreground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,236e9946-7801-4898-b609-06c8ab1139dc,com.kmong.iOS,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Apple,iPhone,iOS,11.4.1,4.0.4,goal,buyer,buyer_order_track,view,NaN,NaN,viral,web,NaN,NaN,NaN,False,False


#### 10. funnel 데이터를 정리

#### 통일성 있는 컬럼명으로 수정하고 알아보기 쉬운 컬럼명으로 수정

In [48]:
raw_funnel.head()

,Lv2,viewid,viewid desc,Lv1,funnel name,funnel desc
0,1100,home,홈 (탭),11,home,홈
1,1210,category_list,카테고리 목록 (탭),12,category,카테고리
2,1200,category_gig,카테고리-상품목록,12,category,카테고리
3,1300,search,검색,13,search,검색
4,1301,search_gig,검색-상품목록,13,search,검색


In [49]:
raw_funnel.rename(columns= {"Lv2" : "lv2", 
                             "viewid" : "view_id", 
                             "viewid desc" : "viewid_desc",
                             "Lv1": "lv1",
                             "funnel name":"funnel_name",
                             "funnel desc":"funnel_desc",
                            }, inplace=True )

print(raw_funnel.shape)

raw_funnel.head()

(53, 6)


,lv2,view_id,viewid_desc,lv1,funnel_name,funnel_desc
0,1100,home,홈 (탭),11,home,홈
1,1210,category_list,카테고리 목록 (탭),12,category,카테고리
2,1200,category_gig,카테고리-상품목록,12,category,카테고리
3,1300,search,검색,13,search,검색
4,1301,search_gig,검색-상품목록,13,search,검색


In [50]:
raw_funnel = pd.DataFrame(raw_funnel, columns = ["lv1", "lv2", "view_id", "viewid_desc", "funnel_name", "funnel_desc"])

print(raw_funnel.shape)

raw_funnel.head()

(53, 6)


,lv1,lv2,view_id,viewid_desc,funnel_name,funnel_desc
0,11,1100,home,홈 (탭),home,홈
1,12,1210,category_list,카테고리 목록 (탭),category,카테고리
2,12,1200,category_gig,카테고리-상품목록,category,카테고리
3,13,1300,search,검색,search,검색
4,13,1301,search_gig,검색-상품목록,search,검색


#### 11. category 데이터를 정리

In [51]:
raw_category.head()

,depth,categoryid,categoryname,cat1_id,cat2_id,cat3_id,cat1,cat2,cat3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN
1,1,2,마케팅,2,NaN,NaN,마케팅,NaN,NaN
2,1,3,번역·통역,3,NaN,NaN,번역·통역,NaN,NaN
3,1,4,문서작성,4,NaN,NaN,문서작성,NaN,NaN
4,1,6,IT·프로그래밍,6,NaN,NaN,IT·프로그래밍,NaN,NaN


In [52]:
raw_category.rename(columns= {"categoryid" : "category_id", 
                             "categoryname" : "category_name",
                             "cat1_id": "category1_id",
                             "cat2_id":"category2_id",
                             "cat3_id":"category3_id",
                             "cat1":"category1",
                             "cat2":"category2",
                             "cat3":"category3",
                            }, inplace=True )

print(raw_category.shape)

raw_category.head()

(245, 9)


,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN
1,1,2,마케팅,2,NaN,NaN,마케팅,NaN,NaN
2,1,3,번역·통역,3,NaN,NaN,번역·통역,NaN,NaN
3,1,4,문서작성,4,NaN,NaN,문서작성,NaN,NaN
4,1,6,IT·프로그래밍,6,NaN,NaN,IT·프로그래밍,NaN,NaN


### 데이터를 하나로 합치기

#### 12. dropped_log 와 raw_funnel 합치기


In [53]:
data = pd.merge(dropped_log, raw_funnel, on='view_id', how='left')

print(data.shape)

data.head()

(434244, 33)


,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,viewid_desc,funnel_name,funnel_desc
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,NaN,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,11.0,1100.0,홈 (탭),home,홈
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,NaN,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14.0,1400.0,상품상세,gig,상품
2,14eb3197-db83-493a-b7be-83582960c40b,com.kmong.iOS,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,12.0,4.0.4,goal,inbox,inbox_detail,view,NaN,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,16.0,1610.0,메시지목록-상세,inbox,메시지
3,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,com.kmong.iOS,168761CB-CB67-4592-867D-52780D651297,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1,Apple,iPhone,iOS,11.4.1,4.0.4,foreground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,236e9946-7801-4898-b609-06c8ab1139dc,com.kmong.iOS,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Apple,iPhone,iOS,11.4.1,4.0.4,goal,buyer,buyer_order_track,view,NaN,NaN,viral,web,NaN,NaN,NaN,False,False,24.0,2410.0,메뉴목록-구매관리-거래메시지,transaction_history,거래관리


In [54]:
data[data["row_uuid"] == "e62dccef-dd70-4415-8a33-c8324ddaed38"]

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,viewid_desc,funnel_name,funnel_desc
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,NaN,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14.0,1400.0,상품상세,gig,상품


#### 13. data 와 raw_category 합치기

In [55]:
raw_category.head()

,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
0,1,1,디자인,1,NaN,NaN,디자인,NaN,NaN
1,1,2,마케팅,2,NaN,NaN,마케팅,NaN,NaN
2,1,3,번역·통역,3,NaN,NaN,번역·통역,NaN,NaN
3,1,4,문서작성,4,NaN,NaN,문서작성,NaN,NaN
4,1,6,IT·프로그래밍,6,NaN,NaN,IT·프로그래밍,NaN,NaN


In [56]:
data = pd.merge(data, raw_category, how='left', left_on='in_app_event_label', right_on = 'category_id')

print(data.shape)

data.head()

(434244, 42)


,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,viewid_desc,funnel_name,funnel_desc,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,NaN,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,11.0,1100.0,홈 (탭),home,홈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,NaN,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14.0,1400.0,상품상세,gig,상품,3.0,41201.0,자기소개서,4.0,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
2,14eb3197-db83-493a-b7be-83582960c40b,com.kmong.iOS,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,12.0,4.0.4,goal,inbox,inbox_detail,view,NaN,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,16.0,1610.0,메시지목록-상세,inbox,메시지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,com.kmong.iOS,168761CB-CB67-4592-867D-52780D651297,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1,Apple,iPhone,iOS,11.4.1,4.0.4,foreground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,236e9946-7801-4898-b609-06c8ab1139dc,com.kmong.iOS,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Apple,iPhone,iOS,11.4.1,4.0.4,goal,buyer,buyer_order_track,view,NaN,NaN,viral,web,NaN,NaN,NaN,False,False,24.0,2410.0,메뉴목록-구매관리-거래메시지,transaction_history,거래관리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
data[data["row_uuid"] == "e62dccef-dd70-4415-8a33-c8324ddaed38"]

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,viewid_desc,funnel_name,funnel_desc,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,NaN,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14.0,1400.0,상품상세,gig,상품,3.0,41201.0,자기소개서,4.0,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서


#### 14. 필요하지 않은 컬럼버리고 정리

In [58]:
data.head()

,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,in_app_event_category,in_app_event_label,source_type,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity,lv1,lv2,viewid_desc,funnel_name,funnel_desc,depth,category_id,category_name,category1_id,category2_id,category3_id,category1,category2,category3
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,NaN,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,11.0,1100.0,홈 (탭),home,홈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,NaN,41201.0,unattributed,unattributed,NaN,NaN,NaN,False,False,14.0,1400.0,상품상세,gig,상품,3.0,41201.0,자기소개서,4.0,412.0,41201.0,문서작성,자기소개서·이력서,자기소개서
2,14eb3197-db83-493a-b7be-83582960c40b,com.kmong.iOS,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,12.0,4.0.4,goal,inbox,inbox_detail,view,NaN,NaN,unattributed,unattributed,NaN,NaN,NaN,False,False,16.0,1610.0,메시지목록-상세,inbox,메시지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,com.kmong.iOS,168761CB-CB67-4592-867D-52780D651297,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1,Apple,iPhone,iOS,11.4.1,4.0.4,foreground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,236e9946-7801-4898-b609-06c8ab1139dc,com.kmong.iOS,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Apple,iPhone,iOS,11.4.1,4.0.4,goal,buyer,buyer_order_track,view,NaN,NaN,viral,web,NaN,NaN,NaN,False,False,24.0,2410.0,메뉴목록-구매관리-거래메시지,transaction_history,거래관리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
new_columns = ['row_uuid', 'app_package_name', 'user_id', 'event_datetime', 'event_datetime_year', 'event_datetime_month', 'event_datetime_day',
    'event_datetime_hour', 'event_datetime_minute', 'event_datetime_second', 'device_manufacturer', 'device_type', 'os_type', 'os_version',
    'app_version', 'event_category', 'view_category', 'view_id', 'view_action', 'funnel_desc', 'category_name', 'category1', 'category2', 'category3',
    'channel', 'params_campaign', 'params_medium', 'params_term', 'is_first_activity', 'is_first_goal_activity']

data = data[new_columns].copy()

print(data.shape)

data.head()

(434244, 30)


,row_uuid,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,funnel_desc,category_name,category1,category2,category3,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity
0,fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,홈,NaN,NaN,NaN,NaN,unattributed,NaN,NaN,NaN,False,False
1,e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,상품,자기소개서,문서작성,자기소개서·이력서,자기소개서,unattributed,NaN,NaN,NaN,False,False
2,14eb3197-db83-493a-b7be-83582960c40b,com.kmong.iOS,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,12.0,4.0.4,goal,inbox,inbox_detail,view,메시지,NaN,NaN,NaN,NaN,unattributed,NaN,NaN,NaN,False,False
3,f9bb91af-248b-44dc-9f5c-1c00b37ea97b,com.kmong.iOS,168761CB-CB67-4592-867D-52780D651297,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1,Apple,iPhone,iOS,11.4.1,4.0.4,foreground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,236e9946-7801-4898-b609-06c8ab1139dc,com.kmong.iOS,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Apple,iPhone,iOS,11.4.1,4.0.4,goal,buyer,buyer_order_track,view,거래관리,NaN,NaN,NaN,NaN,web,NaN,NaN,NaN,False,False


#### 15. row_uuid를 인덱스(index)로 지정

In [61]:
# Write your code here!
data = data.set_index("row_uuid")

print(data.shape)

data.head()

(434244, 29)


,app_package_name,user_id,event_datetime,event_datetime_year,event_datetime_month,event_datetime_day,event_datetime_hour,event_datetime_minute,event_datetime_second,device_manufacturer,device_type,os_type,os_version,app_version,event_category,view_category,view_id,view_action,funnel_desc,category_name,category1,category2,category3,channel,params_campaign,params_medium,params_term,is_first_activity,is_first_goal_activity
row_uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
fd2a188c-bc9b-4702-9c47-b546b2614817,com.kmong.iOS,F36FAA62-ADAC-4AA5-9B00-1FD6CB7EE957,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,11.4.1,4.0.4,goal,home,home,view,홈,NaN,NaN,NaN,NaN,unattributed,NaN,NaN,NaN,False,False
e62dccef-dd70-4415-8a33-c8324ddaed38,com.kmong.kmong,8a871e50-0717-4aed-9bad-04ac3c3793be,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Samsung,SM-N935S,Android,7.0,3.3.5,goal,gig,gig_detail,view,상품,자기소개서,문서작성,자기소개서·이력서,자기소개서,unattributed,NaN,NaN,NaN,False,False
14eb3197-db83-493a-b7be-83582960c40b,com.kmong.iOS,A9E5778A-8F3D-4597-9718-74BF953A9F64,2018-09-28 00:00:00+09:00,2018,9,28,0,0,0,Apple,iPhone,iOS,12.0,4.0.4,goal,inbox,inbox_detail,view,메시지,NaN,NaN,NaN,NaN,unattributed,NaN,NaN,NaN,False,False
f9bb91af-248b-44dc-9f5c-1c00b37ea97b,com.kmong.iOS,168761CB-CB67-4592-867D-52780D651297,2018-09-28 00:00:01+09:00,2018,9,28,0,0,1,Apple,iPhone,iOS,11.4.1,4.0.4,foreground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236e9946-7801-4898-b609-06c8ab1139dc,com.kmong.iOS,ACABB7C0-4C76-413A-B314-E5D6DA0D0E5D,2018-09-28 00:00:02+09:00,2018,9,28,0,0,2,Apple,iPhone,iOS,11.4.1,4.0.4,goal,buyer,buyer_order_track,view,거래관리,NaN,NaN,NaN,NaN,web,NaN,NaN,NaN,False,False
